# Exploring links between Officer Carter's arrests and restraint incidents

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import datetime as dt
import matplotlib.pyplot as plt
import altair as alt

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

### Import restraints incidents involving Carter

In [4]:
restraint_df = pd.read_csv(
    "data/processed/carter_restraint_cases.csv",
    dtype={"id": str},
    parse_dates=["date"],
)

### A list of dates in which Carter is mentioned in a restraint narrative

In [5]:
restraint_dates = list(restraint_df.date.unique())

### How many dates? 

In [6]:
len(restraint_dates)

12

In [7]:
restraint_df

,id,date,time,shift,reporting_personnel,number_assisting,location,year,quarter,day,month,weekday,monthname,month_year,month_year_full,hour,minute,time_full,narrative,carter
0,201800138,2018-09-24,30,NITE,"BETANCOURT, L",2.0,BOOKING AREA,2018,3,24,9,Monday,September,2018-09,2018-09-01,0,30,00:30:00,"At aproximately 0030 prisoner Prince,Travis was brougth in to the jail. The prisoner constantly used verbally assaultive language towards officers and jail staff. The prisoner was padded down and placed in the holding cage where he began to hit the cage and said he was going to give the jail staff hell. The prisoner was advised to calm down and to take a seat and wait to be proccesed, but began to jump on the bench. Immediately Ofc Carter and myself PSO Betancourt walked in to the holding cage and told the prisoner to place his hands behind his back. Right away we took control of the prisoner holding his arms behind his back and walked him to the restraint chair. The prisoner was strapped down to the chair by Ofc Carter, PSO Pina, PSO Betancourt, with the prisoner giving some resistance and placed into cell 1708 to be further observed. The prisoner continued to use abusive language and threaten staff as they passed by the cell. PSO Walls was present in the jail when the incident took place, and there was no injuries. [09/24/2018 02:51, LBETANCOURT, PA , PPD]",True
1,201800145,2018-10-05,510,NITE,"BETANCOURT, L",2.0,BOOKING AREA,2018,4,5,10,Friday,October,2018-10,2018-10-01,5,10,05:10:00,"At aproximately 0510 prisoner Medellin,Jesus was placed in the restraint chair after several attempts to reason with the prisoner to calm down. The prisoner came in to the jail in a belligerent manner. The prisoner continously made agressive movements and threats towards officers and staff . The prisoner was placed in the restraint chair by myself PSO Betancourt, Ofc Carter, and PSO Valdez and placed in cell 1708. The prisoner continued to make threats and use assaultive langauage towards staff. PSO supervisor Adame was in the jail when the incident took place. [10/05/2018 05:39, LBETANCOURT, PA , PPD]",True
2,201900129,2019-05-02,1830,EVES,"THOMAS, R",2.0,CELL,2019,2,2,5,Thursday,May,2019-05,2019-05-01,18,30,18:30:00,"On this date at approximately 1830hrs the listed inmate was placed in the restraint chair for attempting to harm herself by hitting the door in her cell, playing in the toilet water, and threatening suicide. While on duty and working as the Jailer, I was notified by PSO Teel and Officer Carter that the inmate had stripped off all of her clothes and was hitting the cell door to her cell. During this time I attempted to diffuse the situation by talking to the inmate via the jail intercom system. The inmate insisted on continuing her belligerent behavior and began telling me over the intercom that she would commit suicide if we did not go into the cell and stop her. PSO`s Teel, Herrera, and I then entered cell 1705 and escorted the inmate to restraint chair cell 1707. The inmate was placed in the restraint chair in cell 1707 without further incident or injury. PSO Supervisor Rhodes was notified and approved. It should be noted that prior to entering cell 1705 the inmate had put her clothing back on and was adequately dressed. [05/02/2019 19:43, RTHOMAS, PA , PPD]",True
3,201900150,2019-05-25,1450,EVES,"HERRERA, M",NaN,SALLY PORT,2019,2,25,5,Saturday,May,2019-05,2019-05-01,14,50,14:50:00,"INMATE WAS CHAIRED UPON ARRIVAL DUE TO HIM BEING BELLIGERENT ON SCENE AND SPITTING AT OFFICER CARTERS BOOTS. [05/25/2019 15:16, MHERRERA, PA , PPD]",True
4,201900152,2019-05-29,1930,EVES,"OROZCO, C",2.0,CELL,2019,2,29,5,Wednesday,May,2019-05,2019-05-01,19,30,19:30:00,"Inmate was checked out earlier by ems staff claiming to have extreme pain from spider bites and seizures. Inmate was throwing himself on the floor while bracing his fall and attempted to show signs of a seizure. Inmate was told this was not going to get him out of jail and he then began to 

---

### Import arrests by Carter

In [8]:
arrests_df = pd.read_csv(
    "data/processed/jason_carter_arrests.csv",
    dtype={"case__no": str, "zip_code": str},
    parse_dates=["arrest_date"],
)

### Filter arrests dataframe to dates when Carter is mentioned in restraint logs

In [9]:
arrests_restraints = arrests_df[arrests_df["arrest_date"].isin(restraint_dates)]

In [10]:
arrests_restraints

,case__no,arrest_date,arrest_time,arresting_agency,arresting_officer,arrestee_age,arrestee_sex,arrestee_race,arrestee_ethnicity,arrest_charge,arrest_location,city,state,zip_code,geox,geoy,year,quarter,day,month,weekday,monthname,month_year,month_year_full,arrest_hour,arrest_minute,arrest_time_full,arrest_time_period
15,21005010,2021-04-03,234,PPD,"CARTER, J",40,M,W,H,DRIVING WHILE INTOXICATED - MA & MB,2007 PRESTON AVE NB,PASADENA,TX,77503,3183601.50,13815675,2021,2,3,4,5,April,2021-04,2021-04-01,2,34,02:34:00,1-4am
16,21005044,2021-04-03,1530,PPD,"CARTER, J",41,M,W,H,DL - NO TEXAS DL,1300 PASADENA FWY EB,PASADENA,TX,77506,3175366.00,13826047,2021,2,3,4,5,April,2021-04,2021-04-01,15,30,15:30:00,1-4pm
33,21000942,2021-01-19,1911,PPD,"CARTER, J",40,M,W,H,"AGGRAVATED ASSAULT (FAMILY VIOLENCE), F",1107 W PITTS AVE,PASADENA,TX,77506,3167292.00,13826600,2021,1,19,1,1,January,2021-01,2021-01-01,19,11,19:11:00,5-8pm
34,21000942,2021-01-19,1911,PPD,"CARTER, J",40,M,W,NaN,93OC - OUTSIDE WARRANT (FELONY),1107 W PITTS AVE,PASADENA,TX,77506,3167292.00,13826600,2021,1,19,1,1,January,2021-01,2021-01-01,19,11,19:11:00,5-8pm
57,20016230,2020-11-11,1821,PPD,"CARTER, J",47,M,B,N,TERRORISTIC THREAT - MA & MB,6330 FAIRMONT PKWY EB,PASADENA,TX,77505,3193326.50,13803984,2020,4,11,11,2,November,2020-11,2020-11-01,18,21,18:21:00,5-8pm
135,19022224,2019-12-29,1613,PPD,"CARTER, J",30,M,W,H,"INJURY TO A CHILD/ELDERLY INDIV/DISABLED INDIVIDUAL, F",1808 DADE ST,PASADENA,TX,77502,3170630.00,13816229,2019,4,29,12,6,December,2019-12,2019-12-01,16,13,16:13:00,1-4pm
136,19022224,2019-12-29,1613,PPD,"CARTER, J",30,M,W,NaN,93OC - OUTSIDE WARRANT (FELONY),1808 DADE ST,PASADEN,TX,77502,0.00,0,2019,4,29,12,6,December,2019-12,2019-12-01,16,13,16:13:00,1-4pm
137,19022246,2019-12-29,2259,PPD,"CARTER, J",17,M,W,H,"BRIBERY, F",920 HOUSTON AVE,PASADENA,TX,77502,3174379.50,13816996,2019,4,29,12,6,December,2019-12,2019-12-01,22,59,22:59:00,9-midnight
241,19008795,2019-05-29,1855,PPD,"CARTER, J",22,M,W,H,POSS DRUG PARAPHERNALIA,1705 JENKINS RD,PASADENA,TX,77506,3178166.50,13820353,2019,2,29,5,2,May,2019-05,2019-05-01,18,55,18:55:00,5-8pm
243,19008539,2019-05-25,1454,PPD,"CARTER, J",41,M,W,H,THEFT - SHOPLIFTING - FEL,1000 PASADENA BLVD,PASADENA,TX,77506,3174352.75,13820442,2019,2,25,5,5,May,2019-05,2019-05-01,14,54,14:54:00,1-4pm


---

### Cases/dates in which Carter made an arrest and also is mentioned in restraint narrative

In [11]:
### Date matches and time is close

likely_links = [
    {"id": "202100012", "case__no": "21000942"},
    {"id": "202000321", "case__no": "20016230"},
    {"id": "201900373", "case__no": "19022224"},
    {"id": "201900152", "case__no": "19008795"},
    {"id": "201900150", "case__no": "19008539"},
]

In [12]:
cases_df = pd.DataFrame(likely_links)
cases_df

,id,case__no
0,202100012,21000942
1,202000321,20016230
2,201900373,19022224
3,201900152,19008795
4,201900150,19008539


---

### Merge the linked cases into a single dataframe

In [13]:
arrests_merge = pd.merge(cases_df, arrests_df, on="case__no")

In [14]:
arrests_restraints = pd.merge(arrests_merge, restraint_df, on="id")

In [15]:
len(arrests_restraints)

7

In [16]:
arrests_restraints.to_csv(
    "data/processed/carter_arrest_restraint_linked_date_time.csv", index=False
)